In [6]:
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import pandas as pd
import csv
import calendar

# Function to fetch data from the API for a specific year and month
def fetch_data(year, month, session, url):
    try:
        response = session.get(url, timeout=30)  # Timeout added)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data for {year}-{month}. Status Code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching data for {year}-{month}: {e}")
        return None

In [7]:
# Function to save the data to a CSV file
def save_to_csv(data, filename, col_name_list):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(col_name_list)
        # Write the data
        for entry in data:
            writer.writerow(entry)

In [ ]:
# Function to get all data with retry mechanism
def get_all_data():
    # Create a session with retry mechanism
    session = requests.Session()
    retry = Retry(
        total=5,  # Total number of retry attempts
        backoff_factor=1,  # Time to wait between retries (e.g., 1s, 2s, 4s, etc.)
        status_forcelist=[500, 502, 503, 504],  # HTTP status codes to retry on
        raise_on_status=False,  # Do not raise exceptions for HTTP errors
        allowed_methods=["GET"],  # HTTP methods to apply the retry mechanism
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("https://", adapter)

    all_data = []
    url = f"https://api.data.ipu.org/v1/reports/women-ranking?year={year}&month={month}"
    for year in range(2019, 2026):  # Loop through years
        for month in range(1, 13):  # Loop through months
            print(f"Fetching data for {year}-{month}...")
            result = fetch_data(year, month, session, url)
            if result and "data" in result:
                for _, details in result["data"].items():
                      lower_elec_date = details.get("lower_chamber_election_date") or {}
                      upper_elec_date = details.get("upper_chamber_election_date") or {}
                      all_data.append([
                        year,
                        month,
                        details["country_name"]["en"],
                        details["country_code"],
                        details["region"],
                        details["subregion"],
                        details["lower_chamber_current_members_number"],
                        details["lower_chamber_current_women_number"],
                        details.get("lower_chamber_percent_women", "N/A"),
                        details.get("upper_chamber_percent_women", "N/A"),
                        details.get("rank", "N/A"),
                        # Extract lower chamber election dates with default values
                        lower_elec_date.get("from", "N/A"),
                        lower_elec_date.get("to", "N/A"),
                        # Extract upper chamber election dates with default values
                        upper_elec_date.get("from", "N/A"),
                        upper_elec_date.get("to", "N/A"),
                    ])
    # Save the data to a CSV file
    col_name_list = [
            "Year", "Month", "Country", "Country Code", "Region", "Subregion", 'Total Seat','Total Women',
            "Lower Chamber Women (%)", "Upper Chamber Women (%)", "Rank", "lower_elec_from","lower_elec_to",
            "upper_elec_from","upper_elec_to"
        ]    
    save_to_csv(all_data, "women_representation_2019_2025.csv",col_name_list)
    print("Data collection completed and saved to women_representation_2019_2025.csv.")

# Example usage
if __name__ == "__main__":
    data = get_all_data()
    print("Data retrieval complete. Total entries:", len(data))

Fetching data for 2019-1...
Fetching data for 2019-2...
Fetching data for 2019-3...
Fetching data for 2019-4...
Fetching data for 2019-5...
Fetching data for 2019-6...
Fetching data for 2019-7...
Fetching data for 2019-8...
Fetching data for 2019-9...
Fetching data for 2019-10...
Fetching data for 2019-11...
Fetching data for 2019-12...
Fetching data for 2020-1...
Fetching data for 2020-2...
Fetching data for 2020-3...
Fetching data for 2020-4...
Fetching data for 2020-5...
Fetching data for 2020-6...
Fetching data for 2020-7...
Fetching data for 2020-8...
Fetching data for 2020-9...
Fetching data for 2020-10...
Fetching data for 2020-11...
Fetching data for 2020-12...
Fetching data for 2021-1...
Fetching data for 2021-2...
Fetching data for 2021-3...
Fetching data for 2021-4...
Fetching data for 2021-5...
Fetching data for 2021-6...
Fetching data for 2021-7...
Fetching data for 2021-8...
Fetching data for 2021-9...
Fetching data for 2021-10...
Fetching data for 2021-11...
Fetching dat

TypeError: object of type 'NoneType' has no len()

In [15]:
!pip install --upgrade urllib3 requests

Requirement already up-to-date: requests in c:\users\nuwai\anaconda3\lib\site-packages (2.31.0)
  Found existing installation: urllib3 1.24.1
    Uninstalling urllib3-1.24.1:
      Successfully uninstalled urllib3-1.24.1


In [23]:
# Function to get all data with retry mechanism
def get_all_meta_data():
    # Create a session with retry mechanism
    session = requests.Session()
    retry = Retry(
        total=5,  # Total number of retry attempts
        backoff_factor=1,  # Time to wait between retries (e.g., 1s, 2s, 4s, etc.)
        status_forcelist=[500, 502, 503, 504],  # HTTP status codes to retry on
        raise_on_status=False,  # Do not raise exceptions for HTTP errors
        allowed_methods=["GET"],  # HTTP methods to apply the retry mechanism
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("https://", adapter)

    all_data = []
   
    for year in range(2019, 2026):  # Loop through years
        for month in range(1, 13):  # Loop through months
            # Calculate the last day of the month
            last_day = calendar.monthrange(year, month)[1]
            # Construct the URL
            url = (
                f"https://api.data.ipu.org/v1/explorer?"
                f"fields=country_code,country_name,is_suspended_chamber,structure_of_parliament,region,"
                f"political_system,subregion,is_transitional,ipu_membership,specialized_body__gender_equality,"
                f"themes,is_electoral_quota_women&language=en&sort=country_name.en&page[size]=25&page[number]=1"
                f"&date_range[from]={year}-{month:02d}-01&date_range[to]={year}-{month:02d}-{last_day:02d}"
            )
            print(f"Fetching data for {year}-{month}...")
            result = fetch_data(year, month, session, url)
            if result and "data" in result:
                for _, details in result["data"].items():
                    all_data.append([
                    year,
                    month,
                    details["country_name"]["value"]["en"],
                    details["is_suspended_chamber"]["value"],
                    details["structure_of_parliament"]["value"]["term"],
                    details["region"]["value"]["term"],
                    details["political_system"]["value"]["term"],
                    details["subregion"]["value"]["term"],
                    details["is_transitional"]["value"],
                    details["ipu_membership"]["value"],
                    details["specialized_body__gender_equality"]["value"]["term"] if details["specialized_body__gender_equality"]["value"] else "N/A",
                    details["themes"]["value"][0]["term"] if details["themes"]["value"] and isinstance(details["themes"]["value"], list) and len(details["themes"]["value"]) > 0 else "N/A",
                    details["is_electoral_quota_women"]["value"] 
                ])
    # Save the data to a CSV file
    headers = [
        "Year", "Month", "Country Name", "Suspended Parliament", "Structure of Parliament",
        "Region", "Political System", "Sub-region", "Transitional Status",
        "IPU Membership", "Gender Equality", "Themes", "Gender Quota"
    ]
    save_to_csv(all_data, "ipu_wip_meta_data.csv",headers)
    print("Data collection completed and saved to ipu_wip_meta_data.csv.")

# Example usage
if __name__ == "__main__":
    data = get_all_meta_data()
    print("Data retrieval complete. Total entries:")

Fetching data for 2019-1...
Fetching data for 2019-2...
Fetching data for 2019-3...
Fetching data for 2019-4...
Fetching data for 2019-5...
Fetching data for 2019-6...
Fetching data for 2019-7...
Fetching data for 2019-8...
Fetching data for 2019-9...
Fetching data for 2019-10...
Fetching data for 2019-11...
Fetching data for 2019-12...
Fetching data for 2020-1...
Fetching data for 2020-2...
Fetching data for 2020-3...
Fetching data for 2020-4...
Fetching data for 2020-5...
Fetching data for 2020-6...
Fetching data for 2020-7...
Fetching data for 2020-8...
Fetching data for 2020-9...
Fetching data for 2020-10...
Fetching data for 2020-11...
Fetching data for 2020-12...
Fetching data for 2021-1...
Fetching data for 2021-2...
Fetching data for 2021-3...
Fetching data for 2021-4...
Fetching data for 2021-5...
Fetching data for 2021-6...
Fetching data for 2021-7...
Fetching data for 2021-8...
Fetching data for 2021-9...
Fetching data for 2021-10...
Fetching data for 2021-11...
Fetching dat

'womens_caucus'